# 🛡️ KAAL Offline RL Training Notebook

## RAKSHAK - Agentic AI Cyber Guardian

This notebook trains the **KAAL (Knowledge-Augmented Autonomous Learner)** Dueling DQN agent for autonomous cyber defense.

### What is KAAL?
KAAL is a reinforcement learning agent that decides defensive actions against cyber threats:
- **MONITOR** - Continue observing the threat
- **DEPLOY_HONEYPOT** - Deploy a decoy to gather intelligence
- **ISOLATE_DEVICE** - Quarantine the compromised device
- **ENGAGE_ATTACKER** - Redirect attacker to honeypot
- **ALERT_USER** - Send notification to user

### Training Approach
- **Offline RL**: Train from stored attack events (no live environment needed)
- **Dueling DQN**: Separates value and advantage for better learning
- **Experience Replay**: Random sampling for stable training

### Output
- `kaal_policy.pth` - Trained model for Jetson deployment

---
**Author**: Team RAKSHAK  
**Runtime**: GPU recommended (T4 or better)

## 1️⃣ Setup & Dependencies

In [ ]:
# Install dependencies
!pip install torch numpy matplotlib tqdm -q

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from dataclasses import dataclass
from typing import List, Dict, Any, Optional
from datetime import datetime, timedelta
import random
import json
import os
from tqdm.auto import tqdm

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Using device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Mount Google Drive (for saving model)
from google.colab import drive
drive.mount('/content/drive')

# Create output directory
OUTPUT_DIR = '/content/drive/MyDrive/RAKSHAK_Models'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"📁 Output directory: {OUTPUT_DIR}")

## 2️⃣ Dueling DQN Architecture

In [ ]:
class DuelingDQN(nn.Module):
    """
    Dueling Deep Q-Network architecture.
    
    Separates the network into value and advantage streams:
    Q(s,a) = V(s) + (A(s,a) - mean(A(s,a)))
    
    This helps the agent learn which states are valuable
    without having to learn the effect of each action.
    """
    
    def __init__(self, state_size: int = 10, action_size: int = 5, hidden_size: int = 128):
        super().__init__()
        
        self.state_size = state_size
        self.action_size = action_size
        
        # Shared feature extraction
        self.feature = nn.Sequential(
            nn.Linear(state_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU()
        )
        
        # Value stream - estimates V(s)
        self.value_stream = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, 1)
        )
        
        # Advantage stream - estimates A(s, a)
        self.advantage_stream = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, action_size)
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        features = self.feature(x)
        value = self.value_stream(features)
        advantage = self.advantage_stream(features)
        
        # Combine: Q(s,a) = V(s) + (A(s,a) - mean(A))
        q_values = value + (advantage - advantage.mean(dim=1, keepdim=True))
        return q_values


@dataclass
class Transition:
    """RL transition tuple."""
    state: np.ndarray
    action: int
    reward: float
    next_state: np.ndarray
    done: bool


class ReplayBuffer:
    """Experience replay buffer."""
    
    def __init__(self, capacity: int = 100000):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, transition: Transition):
        self.buffer.append(transition)
    
    def sample(self, batch_size: int) -> List[Transition]:
        return random.sample(self.buffer, min(batch_size, len(self.buffer)))
    
    def __len__(self) -> int:
        return len(self.buffer)


# Test the architecture
model = DuelingDQN().to(device)
test_input = torch.randn(1, 10).to(device)
test_output = model(test_input)
print(f"✅ DuelingDQN created successfully!")
print(f"   Input shape: {test_input.shape}")
print(f"   Output shape: {test_output.shape}")
print(f"   Q-values: {test_output.detach().cpu().numpy().flatten()}")

## 3️⃣ Training Data

Choose one of the options below:
- **Option A**: Generate synthetic training data (for testing)
- **Option B**: Upload real events from Jetson

In [ ]:
# =============================================================================
# OPTION A: Generate Synthetic Training Data
# =============================================================================
# Use this for testing the training pipeline

def generate_synthetic_events(num_events: int = 5000) -> List[Dict]:
    """
    Generate synthetic attack events for training.
    
    Uses SEVERITY-BASED optimal action selection to train the model
    to respond appropriately based on threat severity.
    """
    ATTACK_TYPES = ['port_scan', 'brute_force', 'dos_attack', 'malware', 
                    'exploit_attempt', 'data_exfiltration', 'unauthorized_access']
    SEVERITIES = ['low', 'medium', 'high', 'critical']
    ACTIONS = ['MONITOR', 'DEPLOY_HONEYPOT', 'ISOLATE_DEVICE', 'ENGAGE_ATTACKER', 'ALERT_USER']
    
    # SEVERITY-BASED optimal action mapping (KEY FIX!)
    # Critical/High threats -> ISOLATE immediately
    # Medium threats -> ENGAGE to gather intel
    # Low threats -> MONITOR or deploy honeypot
    OPTIMAL_BY_SEVERITY = {
        'critical': ['ISOLATE_DEVICE'],
        'high': ['ISOLATE_DEVICE', 'ENGAGE_ATTACKER'],
        'medium': ['ENGAGE_ATTACKER', 'DEPLOY_HONEYPOT'],
        'low': ['MONITOR', 'DEPLOY_HONEYPOT']
    }
    
    events = []
    base_time = datetime.now()
    
    for i in range(num_events):
        attack_type = random.choice(ATTACK_TYPES)
        severity = random.choice(SEVERITIES)
        
        # Severity affects state vector
        severity_val = SEVERITIES.index(severity) / 3.0
        attack_val = ATTACK_TYPES.index(attack_type) / 7.0
        
        # Generate realistic state vector
        state = [
            attack_val,                           # attack_type
            severity_val,                         # severity (0=low, 1=critical)
            random.random(),                      # source_port (normalized)
            random.random() * 0.2,                # target_port (typically low)
            random.random() * severity_val,       # packets_per_sec
            random.random(),                      # duration
            1.0 if random.random() > 0.7 else 0.0,  # is_known_attacker
            random.random() * 0.5 + severity_val * 0.5,  # device_risk
            random.random(),                      # time_of_day
            random.random() * 0.3 + 0.2          # protocol_risk
        ]
        
        # Choose action based on SEVERITY (not attack type)
        if random.random() > 0.3:
            # 70% of time: choose severity-optimal action
            action = random.choice(OPTIMAL_BY_SEVERITY.get(severity, ACTIONS))
        else:
            # 30% exploration: random action
            action = random.choice(ACTIONS)
        
        action_id = ACTIONS.index(action)
        
        # Outcome depends on severity-action match
        optimal = action in OPTIMAL_BY_SEVERITY.get(severity, [])
        success = optimal or random.random() > 0.5
        
        events.append({
            'event_id': f'evt-{i:05d}',
            'timestamp': (base_time - timedelta(minutes=i*5)).isoformat(),
            'source_ip': f'192.168.1.{random.randint(100, 200)}',
            'target_ip': '192.168.1.1',
            'attack_type': attack_type,
            'severity': severity,
            'state_vector': state,
            'action_taken': action,
            'action_id': action_id,
            'outcome': 'blocked' if action == 'ISOLATE_DEVICE' else 'engaged',
            'outcome_success': success,
            'metadata': {'ttp_captured': success and action == 'ENGAGE_ATTACKER'}
        })
    
    return events

# Generate events
USE_SYNTHETIC = True  # Set to False if uploading real events

if USE_SYNTHETIC:
    events = generate_synthetic_events(10000)
    print(f"✅ Generated {len(events)} synthetic events")
    print(f"   Sample event: {events[0]['attack_type']} ({events[0]['severity']}) -> {events[0]['action_taken']}")
    
    # Show severity distribution
    from collections import Counter
    sev_counts = Counter([e['severity'] for e in events])
    print(f"\n📊 Severity Distribution:")
    for sev in ['low', 'medium', 'high', 'critical']:
        print(f"   {sev}: {sev_counts[sev]} ({100*sev_counts[sev]/len(events):.1f}%)")

In [ ]:
# =============================================================================
# OPTION B: Upload Real Events from Jetson
# =============================================================================
# Run this cell if you have real events exported from RAKSHAK

UPLOAD_EVENTS = False  # Set to True to upload

if UPLOAD_EVENTS:
    from google.colab import files
    
    print("📤 Upload your events JSON file(s):")
    uploaded = files.upload()
    
    events = []
    for filename, content in uploaded.items():
        data = json.loads(content.decode('utf-8'))
        if isinstance(data, dict) and 'events' in data:
            events.extend(data['events'])
        elif isinstance(data, list):
            events.extend(data)
        print(f"   Loaded {len(events)} events from {filename}")
    
    print(f"\n✅ Total events loaded: {len(events)}")

## 4️⃣ Reward Computation & Transition Building

In [ ]:
def compute_reward(event: Dict) -> float:
    """
    Compute reward based on SEVERITY-ACTION coupling.
    
    Key insight: The optimal action depends on threat SEVERITY, not just attack type.
    - Critical threats -> ISOLATE immediately (no time to gather intel)
    - High threats -> ISOLATE or ENGAGE
    - Medium threats -> Best for intel gathering (ENGAGE/HONEYPOT)
    - Low threats -> MONITOR to conserve resources
    
    This prevents policy collapse toward a single action.
    """
    severity = event.get('severity', 'medium')
    action = event.get('action_taken', 'MONITOR')
    success = event.get('outcome_success', True)
    
    # Severity-action optimal mapping (MUST MATCH generate_synthetic_events!)
    OPTIMAL_BY_SEVERITY = {
        'critical': ['ISOLATE_DEVICE'],
        'high': ['ISOLATE_DEVICE', 'ENGAGE_ATTACKER'],
        'medium': ['ENGAGE_ATTACKER', 'DEPLOY_HONEYPOT'],
        'low': ['MONITOR', 'DEPLOY_HONEYPOT']
    }
    
    # Base reward scales with severity
    BASE_REWARDS = {
        'critical': 10.0,
        'high': 5.0,
        'medium': 2.0,
        'low': 0.5
    }
    
    optimal_actions = OPTIMAL_BY_SEVERITY.get(severity, [])
    is_optimal = action in optimal_actions
    base = BASE_REWARDS.get(severity, 1.0)
    
    if success:
        if is_optimal:
            # High reward for severity-appropriate action
            reward = base * 2.0
            
            # Bonus for TTP capture (only valid for ENGAGE on medium threats)
            if event.get('metadata', {}).get('ttp_captured'):
                reward += 2.0
        else:
            # Suboptimal action - reduced reward
            # Penalize more for critical threats handled incorrectly
            penalty = 0.3 if severity in ['critical', 'high'] else 0.5
            reward = base * penalty
    else:
        # Failed action - negative reward
        reward = -base
    
    return reward


def build_transitions(events: List[Dict]) -> List[Transition]:
    """
    Convert events to RL transitions.
    
    Each event becomes (state, action, reward, next_state, done).
    """
    transitions = []
    
    for i, event in enumerate(events):
        state = event.get('state_vector', [0.0] * 10)
        if len(state) != 10:
            continue
        
        state = np.array(state, dtype=np.float32)
        action = event.get('action_id', 0)
        reward = compute_reward(event)
        
        # Get next state
        if i + 1 < len(events):
            next_state = events[i + 1].get('state_vector', [0.0] * 10)
            done = False
        else:
            next_state = [0.0] * 10
            done = True
        
        next_state = np.array(next_state, dtype=np.float32)
        
        transitions.append(Transition(
            state=state,
            action=action,
            reward=reward,
            next_state=next_state,
            done=done
        ))
    
    return transitions


# Build transitions
transitions = build_transitions(events)
print(f"✅ Built {len(transitions)} transitions")

# Analyze rewards
rewards = [t.reward for t in transitions]
print(f"\n📊 Reward Statistics:")
print(f"   Mean: {np.mean(rewards):.2f}")
print(f"   Std:  {np.std(rewards):.2f}")
print(f"   Min:  {np.min(rewards):.2f}")
print(f"   Max:  {np.max(rewards):.2f}")

# Action distribution
actions = [t.action for t in transitions]
action_names = ['MONITOR', 'DEPLOY_HONEYPOT', 'ISOLATE_DEVICE', 'ENGAGE_ATTACKER', 'ALERT_USER']
print(f"\n📊 Action Distribution:")
for i, name in enumerate(action_names):
    count = actions.count(i)
    pct = 100*count/len(actions) if len(actions) > 0 else 0
    print(f"   {name}: {count} ({pct:.1f}%)")

# Show reward by action (should be balanced now!)
print(f"\n📊 Average Reward by Action:")
for i, name in enumerate(action_names):
    action_rewards = [t.reward for t in transitions if t.action == i]
    if action_rewards:
        print(f"   {name}: {np.mean(action_rewards):.2f}")

## 5️⃣ Training Loop

In [ ]:
# =============================================================================
# Training Configuration
# =============================================================================

CONFIG = {
    'epochs': 200,           # Number of training epochs
    'batch_size': 64,        # Batch size
    'learning_rate': 0.001,  # Learning rate
    'gamma': 0.99,           # Discount factor
    'target_update': 10,     # Update target network every N epochs
    'state_size': 10,        # State vector size
    'action_size': 5,        # Number of actions
    'hidden_size': 128       # Hidden layer size
}

print("⚙️ Training Configuration:")
for k, v in CONFIG.items():
    print(f"   {k}: {v}")

In [ ]:
def train_kaal(
    transitions: List[Transition],
    config: Dict,
    device: torch.device
) -> tuple:
    """
    Train KAAL Dueling DQN using offline experience replay.
    
    Returns:
        (policy_net, losses, best_loss)
    """
    # Initialize networks
    policy_net = DuelingDQN(
        config['state_size'],
        config['action_size'],
        config['hidden_size']
    ).to(device)
    
    target_net = DuelingDQN(
        config['state_size'],
        config['action_size'],
        config['hidden_size']
    ).to(device)
    target_net.load_state_dict(policy_net.state_dict())
    target_net.eval()
    
    # Optimizer
    optimizer = optim.Adam(policy_net.parameters(), lr=config['learning_rate'])
    
    # Load transitions into replay buffer
    replay_buffer = ReplayBuffer(capacity=len(transitions) + 1000)
    for t in transitions:
        replay_buffer.push(t)
    
    print(f"\n🚀 Starting Training...")
    print(f"   Transitions: {len(replay_buffer)}")
    print(f"   Epochs: {config['epochs']}")
    print(f"   Batch size: {config['batch_size']}")
    
    losses = []
    best_loss = float('inf')
    
    # Training loop
    progress = tqdm(range(config['epochs']), desc="Training")
    
    for epoch in progress:
        epoch_losses = []
        steps_per_epoch = max(1, len(transitions) // config['batch_size'])
        
        for step in range(steps_per_epoch):
            # Sample batch
            batch = replay_buffer.sample(config['batch_size'])
            
            # Prepare tensors
            states = torch.FloatTensor([t.state for t in batch]).to(device)
            actions = torch.LongTensor([t.action for t in batch]).to(device)
            rewards = torch.FloatTensor([t.reward for t in batch]).to(device)
            next_states = torch.FloatTensor([t.next_state for t in batch]).to(device)
            dones = torch.FloatTensor([float(t.done) for t in batch]).to(device)
            
            # Compute Q(s, a)
            current_q = policy_net(states).gather(1, actions.unsqueeze(1))
            
            # Double DQN: select actions with policy, evaluate with target
            with torch.no_grad():
                next_actions = policy_net(next_states).argmax(1, keepdim=True)
                next_q = target_net(next_states).gather(1, next_actions).squeeze(1)
                target_q = rewards + (1 - dones) * config['gamma'] * next_q
            
            # Huber loss for stability
            loss = F.smooth_l1_loss(current_q.squeeze(), target_q)
            
            # Optimize
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(policy_net.parameters(), 1.0)
            optimizer.step()
            
            epoch_losses.append(loss.item())
        
        # Epoch stats
        avg_loss = np.mean(epoch_losses)
        losses.append(avg_loss)
        
        # Update target network
        if (epoch + 1) % config['target_update'] == 0:
            target_net.load_state_dict(policy_net.state_dict())
        
        # Track best
        if avg_loss < best_loss:
            best_loss = avg_loss
        
        # Update progress bar
        progress.set_postfix({'loss': f'{avg_loss:.4f}', 'best': f'{best_loss:.4f}'})
    
    return policy_net, losses, best_loss


# Train!
policy_net, losses, best_loss = train_kaal(transitions, CONFIG, device)

print(f"\n✅ Training Complete!")
print(f"   Final Loss: {losses[-1]:.4f}")
print(f"   Best Loss: {best_loss:.4f}")

In [ ]:
# Plot training loss
plt.figure(figsize=(10, 4))
plt.plot(losses, 'b-', alpha=0.7)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('KAAL Training Loss')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Plot smoothed loss
window = 10
smoothed = np.convolve(losses, np.ones(window)/window, mode='valid')
plt.figure(figsize=(10, 4))
plt.plot(smoothed, 'g-', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Smoothed Loss')
plt.title(f'KAAL Training Loss (smoothed, window={window})')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6️⃣ Save & Download Model

In [ ]:
# Save checkpoint
checkpoint = {
    'policy_state_dict': policy_net.state_dict(),
    'state_size': CONFIG['state_size'],
    'action_size': CONFIG['action_size'],
    'hidden_size': CONFIG['hidden_size'],
    'training_info': {
        'epochs': CONFIG['epochs'],
        'batch_size': CONFIG['batch_size'],
        'learning_rate': CONFIG['learning_rate'],
        'gamma': CONFIG['gamma'],
        'transitions': len(transitions),
        'final_loss': losses[-1],
        'best_loss': best_loss,
        'timestamp': datetime.now().isoformat()
    }
}

# Save to Google Drive
model_path = f"{OUTPUT_DIR}/kaal_policy.pth"
torch.save(checkpoint, model_path)
print(f"✅ Model saved to Google Drive: {model_path}")

# Also save locally for download
local_path = '/content/kaal_policy.pth'
torch.save(checkpoint, local_path)
print(f"✅ Model saved locally: {local_path}")

# Save inference-only version (smaller)
inference_checkpoint = {
    'policy_state_dict': policy_net.state_dict(),
    'state_size': CONFIG['state_size'],
    'action_size': CONFIG['action_size']
}
inference_path = f"{OUTPUT_DIR}/kaal_policy_inference.pth"
torch.save(inference_checkpoint, inference_path)
print(f"✅ Inference model saved: {inference_path}")

In [ ]:
# Download the model
from google.colab import files

print("📥 Downloading model...")
files.download(local_path)

## 7️⃣ Test the Model

In [ ]:
# Test the trained model
ACTIONS = ['MONITOR', 'DEPLOY_HONEYPOT', 'ISOLATE_DEVICE', 'ENGAGE_ATTACKER', 'ALERT_USER']

def test_model(model, state_vector, device):
    """Test model on a state vector."""
    model.eval()
    with torch.no_grad():
        state = torch.FloatTensor(state_vector).unsqueeze(0).to(device)
        q_values = model(state).cpu().numpy().flatten()
        action = np.argmax(q_values)
    return action, q_values


# Test scenarios
test_scenarios = [
    {
        'name': 'Critical DDoS Attack',
        'state': [0.43, 1.0, 0.5, 0.01, 0.9, 0.8, 1.0, 0.9, 0.3, 0.5]
    },
    {
        'name': 'Low Severity Port Scan',
        'state': [0.0, 0.0, 0.3, 0.02, 0.1, 0.1, 0.0, 0.2, 0.5, 0.3]
    },
    {
        'name': 'Brute Force Attack',
        'state': [0.14, 0.67, 0.8, 0.02, 0.5, 0.3, 0.0, 0.5, 0.1, 0.4]
    },
    {
        'name': 'Malware Detected',
        'state': [0.57, 1.0, 0.2, 0.05, 0.3, 0.5, 1.0, 0.8, 0.7, 0.6]
    }
]

print("🧪 Testing Model on Sample Threats")
print("=" * 60)

for scenario in test_scenarios:
    action, q_values = test_model(policy_net, scenario['state'], device)
    print(f"\n📌 {scenario['name']}")
    print(f"   Decision: {ACTIONS[action]}")
    print(f"   Q-values:")
    for i, (name, q) in enumerate(zip(ACTIONS, q_values)):
        marker = "  ←" if i == action else ""
        print(f"      {name:20s}: {q:7.3f}{marker}")

print("\n" + "=" * 60)
print("✅ Model testing complete!")

## 📋 Deployment Instructions

### Copy to Jetson
```bash
scp kaal_policy.pth user@jetson-ip:~/e-raksha/models/
```

### Verify on Jetson
```python
from core.agentic_defender import AgenticDefender
import yaml

with open('config/config.yaml') as f:
    config = yaml.safe_load(f)

# Update config to use new model
config['agent']['model_path'] = 'models/kaal_policy.pth'

agent = AgenticDefender(config)
print(f'Model loaded: {agent.model_loaded}')
print(f'Mode: {agent.get_statistics()["mode"]}')
```

### Run RAKSHAK
```bash
sudo python main.py
```